In [3]:
import pandas as pd
import ipynb.fs.full.features as features_nltk

df_train = pd.read_csv('./data/train.csv')

In [2]:
def concatenar_categoricos(df, columna1, columna2, N=0, df_aux=None, devolver_cols=False):
    '''Concatena dos columnas categoricas y hace one hot'''
    '''Ver referencia columna_a_ohe para los ultimos 3 parametros'''
    
    def limpiar_espacios_blanco(texto):
        return texto.replace(' ', '_')
        
    df = df.copy()
    df[columna1] = df[columna1].apply(limpiar_espacios_blanco)
    df[columna2] = df[columna2].apply(limpiar_espacios_blanco)
    df[columna1 + '_' + columna2] = df[columna1] + '_' +df[columna2]
    
    features_nltk
    return features_nltk.columna_a_ohe(df, columna1 + '_' + columna2, N, df_aux, devolver_cols)

In [88]:
def cantidad_propiedad_misma_cantidad(df, agrupar_col, misma_cantidad_col):
    '''Cantidad de casas con la misma cantidad de banos que la actual'''
    '''Por ejemplo, retorna para cada propiedad, cuantas hay mas de su tipo con x banos ej.'''
    '''cantidad_propiedad_misma_cantidad(df, 'banos', 'tipodepropiedad'), para cada propiedad, se fija
    cuantas mas de su tipo con tantos banos hay'''
    '''Retorna el df con la columna agregada'''
    df_aux = df.copy()
    df_aux = df_aux.groupby([agrupar_col, misma_cantidad_col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()
    
    def get_cantidad(col1, col2):
        cantidad = df_aux[(df_aux[agrupar_col] == col1) & (df_aux[misma_cantidad_col] == col2)]['cantidad']
        return cantidad.values[0] if len(cantidad.values > 0) else 0
        
    df['cantidad_' + agrupar_col + '_' + misma_cantidad_col] = df.apply(lambda x: get_cantidad(x[agrupar_col], x[misma_cantidad_col]), axis=1)
    return df

In [107]:
def cantidad_atributo(df, col):
    '''Simil a value_counts. Dada una columna, agrega al df, cuantas propiedades existen con esa misma propiedad'''
    df_aux = df.copy()
    df_aux = df_aux.groupby([col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()

    def get_cantidad(col1):
        cantidad = df_aux[df_aux[col] == col1]['cantidad']
        return cantidad.values[0] if len(cantidad.values > 0) else 0
    
    df['cantidad_propiedades_en_' + col] = df.apply(lambda x: get_cantidad(x[col]), axis=1)
    return df

In [151]:
def ranking_atributo(df, col):
    '''Cuenta cuantas propiedades hay con los atributos de la columna recibida. Y para cada propiedad,
    se fija en que posicion esta del ranking. Por ejemplo, si hay dos propuedades, una con dos banos y otra 
    con uno solo, el rankin de las que tengan dos banos en 1, y de las propiedades que tengan un solo bano es 
    dos'''
    
    '''Rellena los nulos con "otros" '''
    df[col] = df[col].fillna('otros')
    
    df_aux = df.copy()
    df_aux = df_aux.groupby([col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()
    
    df_aux = df_aux.nlargest(df_aux.shape[0], 'cantidad') 
    lista_ranking = df_aux[col].tolist()
    rank = {}
    contador = 1
    
    for item in lista_ranking:
        rank[item] = contador
        contador = contador + 1

    def get_ranking(col1):
        item = df_aux[df_aux[col] == col1][col].values[0]
        return rank[item]
    
    df['ranking_en_' + col] = df.apply(lambda x: get_ranking(x[col]), axis=1)
    return df

In [153]:
df = df_train.head(30)
# df = df_train

# concatenar_categoricos(df, 'tipodepropiedad', 'provincia')
# cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'provincia')
# cantidad_atributo(df, 'provincia')
ranking_atributo(df, 'provincia')

/home/tomas/Facultad/datos/datos-tp2/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/tomas/Facultad/datos/datos-tp2/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,cantidad_tipodepropiedad_provincia,ranking_en_provincia
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0,3,1
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0,2,1
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0,2,3
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0,2,4
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0,1,3
5,126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,...,-99.148475,2014-03-18 00:00:00,0.0,0.0,0.0,0.0,1.0,1100000.0,3,1
6,139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,NaN,3.0,1.0,...,-96.803504,2016-02-23 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0,1,8
7,5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.0,4.0,2.0,...,-87.037968,2016-10-20 00:00:00,0.0,0.0,0.0,0.0,0.0,4200000.0,1,6
8,44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,...,NaN,2014-01-06 00:00:00,0.0,0.0,0.0,1.0,1.0,310000.0,1,7
9,134537,terreno,"terreno de 5.500m2 bardeado, uso de suelo h-20...",Terreno,Av. Morelos,Ixtapaluca,Edo. de México,NaN,NaN,NaN,...,-98.887000,2016-12-22 00:00:00,0.0,0.0,0.0,0.0,0.0,6200000.0,1,4
